# COMP 432 - BIXI

In [1]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

# Removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Load BIXI data

In [2]:
zf = ZipFile('./data/bixi/biximontrealrentals2019-33ea73.zip')
bixi_2019_04 = pd.read_csv(zf.open('OD_2019-04.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_05 = pd.read_csv(zf.open('OD_2019-05.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_06 = pd.read_csv(zf.open('OD_2019-06.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_07 = pd.read_csv(zf.open('OD_2019-07.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_08 = pd.read_csv(zf.open('OD_2019-08.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_09 = pd.read_csv(zf.open('OD_2019-09.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_10 = pd.read_csv(zf.open('OD_2019-10.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_stations = pd.read_csv(zf.open('Stations_2019.csv'), dtype={'name': 'str'})

/Users/simon/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
bixi_2019_04.head()

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2019-04-14 07:55:22,6001,2019-04-14 08:07:16,6132,713,1
1,2019-04-14 07:59:31,6411,2019-04-14 08:09:18,6411,587,1
2,2019-04-14 07:59:55,6097,2019-04-14 08:12:11,6036,736,1
3,2019-04-14 07:59:57,6310,2019-04-14 08:27:58,6345,1680,1
4,2019-04-14 08:00:37,7029,2019-04-14 08:14:12,6250,814,0


In [4]:
bixi_2019_stations.head()

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478,-73.570
1,4000,Jeanne-d'Arc / Ontario,45.550,-73.542
2,4001,Graham / Brookfield,45.520,-73.630
3,4002,Graham / Wicksteed,45.517,-73.640
4,5002,St-Charles / Montarville,45.534,-73.515


## Load weather data

In [5]:
# Note: Use Date/Time column instead of Time column
weather_2019_04 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_04-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_05 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_05-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_06 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_06-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_07 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_07-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_08 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_08-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_09 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_09-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_10 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_10-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})

In [6]:
weather_2019_04.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 00:00:00,2019,4,1,00:00,-1.500,...,nan,24.100,nan,100.750,nan,nan,nan,-7.000,nan,Snow
1,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 01:00:00,2019,4,1,01:00,-1.700,...,nan,24.100,nan,100.790,nan,nan,nan,-9.000,nan,Cloudy
2,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 02:00:00,2019,4,1,02:00,-2.900,...,nan,24.100,nan,100.860,nan,nan,nan,-9.000,nan,NaN
3,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 03:00:00,2019,4,1,03:00,-3.900,...,nan,24.100,nan,100.920,nan,nan,nan,-10.000,nan,NaN
4,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 04:00:00,2019,4,1,04:00,-4.800,...,nan,24.100,nan,101.030,nan,nan,nan,-11.000,nan,Clear


## Merging all data together

### BIXI data

Merging all 2019 bixi trips together

In [7]:
bixi_2019_all = pd.concat([bixi_2019_04, bixi_2019_05, bixi_2019_06, bixi_2019_07, bixi_2019_08, bixi_2019_09, bixi_2019_10])
bixi_2019_all.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
940801,2019-07-27 14:39:46,7114,2019-07-27 14:56:12,7053,986,0
113548,2019-10-06 12:29:18,6261,2019-10-06 12:54:40,6248,1522,0
714171,2019-06-24 08:58:31,7068,2019-06-24 09:02:33,6087,241,1
92986,2019-05-05 16:05:20,6053,2019-05-05 16:24:27,6407,1146,1
269670,2019-06-09 17:34:10,7105,2019-06-09 17:40:00,7107,349,0


Append station data to all Bixi Trips

***May take a while to run**

In [8]:
bixi_2019_all_with_stations = bixi_2019_all.merge(bixi_2019_stations, left_index=True, left_on='start_station_code', right_on='Code').drop('Code', 1).rename({'name': 'start_name', 'latitude': 'start_latitude', 'longitude': 'start_longitude'}, axis=1).merge(bixi_2019_stations, left_index=True, left_on='end_station_code', right_on='Code').drop('Code', 1).rename({'name': 'end_name', 'latitude': 'end_latitude', 'longitude': 'end_longitude'}, axis=1)
bixi_2019_all_with_stations.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude
366,2019-09-27 10:54:59,7102,2019-09-27 11:05:05,6416,605,0,Grand Boulevard / de Terrebonne,45.471,-73.632,Notre-Dame-de-Grâce / Décarie,45.477,-73.615
94,2019-07-24 13:09:55,6738,2019-07-24 13:22:03,6097,727,1,Union / René-Lévesque,45.502,-73.567,Ste-Catherine / St-Marc,45.493,-73.580
461,2019-06-25 11:00:49,6136,2019-06-25 11:07:07,6926,377,1,Métro Laurier (Rivard / Laurier),45.528,-73.589,Marie-Anne / St-Hubert,45.525,-73.579
430,2019-08-18 22:28:27,6434,2019-08-18 22:52:52,6743,1464,1,Victoria / de Maisonneuve,45.477,-73.600,St-Marc / Sherbrooke,45.495,-73.583
448,2019-07-20 14:46:38,6366,2019-07-20 15:05:59,6908,1160,1,Wilderton / Van Horne,45.510,-73.625,de Bellechasse / de St-Vallier,45.533,-73.601


### Weather

Merging weather data

In [9]:
weather_2019_all = pd.concat([weather_2019_04, weather_2019_05, weather_2019_06, weather_2019_07, weather_2019_08, weather_2019_09, weather_2019_10])
weather_2019_all.sample(10)

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
670,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-28 22:00:00,2019,4,28,22:00,5.600,...,nan,24.100,nan,101.860,nan,nan,nan,nan,nan,Clear
365,-73.740,45.470,MONTREAL INTL A,7025251,2019-07-16 05:00:00,2019,7,16,05:00,18.200,...,nan,24.100,nan,101.130,nan,nan,nan,nan,nan,NaN
657,-73.740,45.470,MONTREAL INTL A,7025251,2019-08-28 09:00:00,2019,8,28,09:00,20.800,...,nan,24.100,nan,100.600,nan,25.000,nan,nan,nan,NaN
310,-73.740,45.470,MONTREAL INTL A,7025251,2019-06-13 22:00:00,2019,6,13,22:00,13.400,...,nan,24.100,nan,100.090,nan,nan,nan,nan,nan,Mostly Cloudy
387,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-17 03:00:00,2019,4,17,03:00,1.400,...,nan,24.100,nan,101.450,nan,nan,nan,nan,nan,NaN
475,-73.740,45.470,MONTREAL INTL A,7025251,2019-07-20 19:00:00,2019,7,20,19:00,26.900,...,nan,24.100,nan,99.960,nan,35.000,nan,nan,nan,Mostly Cloudy
653,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-28 05:00:00,2019,4,28,05:00,2.000,...,nan,24.100,nan,100.830,nan,nan,nan,nan,nan,NaN
530,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-23 02:00:00,2019,4,23,02:00,7.300,...,nan,24.100,nan,101.460,nan,nan,nan,nan,nan,NaN
212,-73.740,45.470,MONTREAL INTL A,7025251,2019-08-09 20:00:00,2019,8,9,20:00,17.700,...,nan,24.100,nan,100.060,nan,nan,nan,nan,nan,NaN
598,-73.740,45.470,MONTREAL INTL A,7025251,2019-05-25 22:00:00,2019,5,25,22:00,15.000,...,nan,24.100,nan,100.500,nan,nan,nan,nan,nan,Cloudy


### BIXI + Weather

Merging weather data (Tempeture and weather) with BIXI data

In [10]:
bixi_2019_all_with_stations_and_weather = bixi_2019_all_with_stations.merge(weather_2019_all[['Date/Time', 'Weather', 'Temp (°C)']], left_on=bixi_2019_all_with_stations['start_date'].dt.floor('H'), right_on='Date/Time').drop('Date/Time', 1).rename({'Weather': 'start_weather', 'Temp (°C)': 'start_temp'}, axis=1)
bixi_2019_all_with_stations_and_weather.sample(10)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude,start_weather,start_temp
4580049,2019-06-05 12:39:36,6070,2019-06-05 12:41:29,6079,112,1,Milton / University,45.506,-73.576,Hutchison / Prince-Arthur,45.510,-73.576,NaN,16.700
4035327,2019-07-06 14:39:52,6079,2019-07-06 14:44:09,7084,257,1,Hutchison / Prince-Arthur,45.510,-73.576,McTavish / Sherbrooke,45.503,-73.576,NaN,29.500
586456,2019-07-08 09:03:30,7014,2019-07-08 09:34:43,6052,1873,1,Métro Université de Montréal (Édouard-Montpeti...,45.504,-73.618,de la Commune / King,45.498,-73.553,NaN,22.200
745935,2019-07-16 08:26:45,6224,2019-07-16 08:31:02,6231,257,1,Villeneuve / du Parc,45.518,-73.592,Jeanne-Mance / St-Viateur,45.523,-73.602,NaN,22.700
3568817,2019-06-21 21:41:38,6014,2019-06-21 21:53:56,6012,738,0,St-Denis / René-Lévesque,45.513,-73.558,Métro St-Laurent (de Maisonneuve / St-Laurent),45.511,-73.565,NaN,20.400
2432833,2019-07-04 19:22:49,6743,2019-07-04 19:43:24,6013,1234,1,St-Marc / Sherbrooke,45.495,-73.583,Sanguinet / de Maisonneuve,45.513,-73.563,Cloudy,30.500
2191959,2019-05-16 17:36:35,7144,2019-05-16 17:55:09,6085,1113,1,Hickson / Wellington,45.465,-73.567,Notre-Dame / Peel,45.495,-73.563,NaN,15.800
4270875,2019-05-17 15:11:01,6058,2019-05-17 15:19:08,6039,487,1,Cypress / Peel,45.499,-73.572,McGill / des Récollets,45.501,-73.559,NaN,12.900
3899013,2019-08-03 09:26:30,6401,2019-08-03 09:32:43,6406,372,1,des Seigneurs / Notre-Dame,45.488,-73.569,Marché Atwater,45.480,-73.578,NaN,23.500
4411784,2019-07-10 10:35:01,6720,2019-07-10 10:40:52,6398,351,1,Ontario / Viau,45.559,-73.535,Desjardins / Ontario,45.551,-73.541,Mostly Cloudy,26.000


## Data Cleaning

### Checking if there are missing data
Only the weather data is missing some values

In [11]:
bixi_2019_all_with_stations_and_weather.isnull().sum()

start_date                  0
start_station_code          0
end_date                    0
end_station_code            0
duration_sec                0
is_member                   0
start_name                  0
start_latitude              0
start_longitude             0
end_name                    0
end_latitude                0
end_longitude               0
start_weather         3459073
start_temp               1494
dtype: int64

### Checking the contents of the "Weather" column
Some of the weather values contains more than 1 condition. We'll have to take this into account while training.

In [12]:
bixi_2019_all_with_stations_and_weather['start_weather'].value_counts()

Mostly Cloudy                              633845
Mainly Clear                               400336
Cloudy                                     304650
Clear                                      281318
Rain                                       165817
Rain Showers                               102038
Rain,Fog                                    20620
Drizzle,Fog                                 17388
Fog                                         16463
Drizzle                                     16140
Thunderstorms,Rain Showers                  11358
Moderate Rain,Fog                            4062
Moderate Rain Showers                        3378
Rain,Rain Showers                            1437
Moderate Rain                                1202
Thunderstorms,Rain                            793
Thunderstorms,Moderate Rain Showers,Fog       635
Thunderstorms,Moderate Rain Showers           612
Thunderstorms,Heavy Rain Showers              577
Thunderstorms                                 274


We will have to find a way to split the weather conditions into smaller parts. 

The best way to do so is to create new features to the data that represent the weather and set the flag to either 0 or 1

Here are the unique weather conditions:
- Mostly Cloudy
- Mainly Clear
- Cloudy
- Clear
- Rain
- Rain Showers
- Fog
- Drizzle
- Moderate Rain Showers
- Moderate Rain
- Heavy Rain Showers
- Thunderstorms

Also, more than half of the data does not have a weather appended to it. I'm not sure if this will affect the results or not. 

## Expanding data

### Weather flags
First, we create new features for the weather data and set the appropriate flag for all trip data

***May take a while to run**

In [13]:
# Using "temp" variable so I can display the results and debug before overwriting the original variable
temp = bixi_2019_all_with_stations_and_weather

temp = temp.join(temp['start_weather'].str.get_dummies(','))

bixi_2019_all_with_stations_and_weather = temp

bixi_2019_all_with_stations_and_weather.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Drizzle,Fog,Heavy Rain Showers,Mainly Clear,Moderate Rain,Moderate Rain Showers,Mostly Cloudy,Rain,Rain Showers,Thunderstorms
4953586,2019-09-29 01:53:13,6709,2019-09-29 02:06:19,6375,786,0,Le Caron / Marc-Sauvalle,45.462,-73.584,Métro Place St-Henri (St-Ferdinand / St-Jacques),...,0,0,0,1,0,0,0,0,0,0
1565323,2019-06-17 17:17:32,6199,2019-06-17 17:22:59,6136,326,1,St-Viateur / St-Laurent,45.526,-73.599,Métro Laurier (Rivard / Laurier),...,0,0,0,0,0,0,0,0,0,0
3752789,2019-09-09 10:30:11,6011,2019-09-09 10:53:06,6143,1374,0,St-André / St-Antoine,45.514,-73.553,Rachel / de Brébeuf,...,0,0,0,1,0,0,0,0,0,0
2251647,2019-06-10 06:06:03,6731,2019-06-10 06:27:38,6262,1294,1,28e avenue / Rosemont,45.564,-73.571,de la Roche / de Bellechasse,...,0,0,0,0,0,0,0,0,0,0
2176651,2019-09-24 16:19:22,6053,2019-09-24 16:44:04,6728,1481,1,du Président-Kennedy / Jeanne-Mance,45.508,-73.569,d'Outremont / Ogilvy,...,0,0,0,0,0,0,0,0,0,0


### Splitting start_date to year, month, day, hour, minute, second columns
Also adding seconds since midnight for simpler normalization

In [14]:
temp = bixi_2019_all_with_stations_and_weather

temp['start_year'] = temp['start_date'].dt.year
temp['start_month'] = temp['start_date'].dt.month
temp['start_day'] = temp['start_date'].dt.day
temp['start_hour'] = temp['start_date'].dt.hour
temp['start_minute'] = temp['start_date'].dt.minute
temp['start_second'] = temp['start_date'].dt.second
temp['start_seconds_since_midnight'] = (temp['start_date'] - pd.to_datetime(temp['start_date'].dt.strftime('%Y-%m-%d') + ' ' + '00:00:00')).astype('timedelta64[s]')

bixi_2019_all_with_stations_and_weather = temp

bixi_2019_all_with_stations_and_weather.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Rain,Rain Showers,Thunderstorms,start_year,start_month,start_day,start_hour,start_minute,start_second,start_seconds_since_midnight
2522063,2019-08-09 08:45:21,6275,2019-08-09 09:01:03,6089,942,1,Chambord / Jean-Talon,45.545,-73.611,Henri-Julien / du Carmel,...,0,0,0,2019,8,9,8,45,21,31521.000
4342161,2019-04-30 22:34:21,6167,2019-04-30 22:42:16,6367,474,1,Marquette / St-Grégoire,45.537,-73.586,3e avenue / Dandurand,...,0,0,0,2019,4,30,22,34,21,81261.000
936226,2019-07-03 08:57:34,6906,2019-07-03 09:05:37,6752,483,1,Métro Rosemont (de St-Vallier / Rosemont),45.532,-73.598,Hutchison / Beaubien,...,0,0,0,2019,7,3,8,57,34,32254.000
2426960,2019-06-17 13:56:11,6099,2019-06-17 14:16:22,6026,1211,0,Ste-Catherine / Labelle,45.515,-73.559,de la Commune / Place Jacques-Cartier,...,0,0,0,2019,6,17,13,56,11,50171.000
4907218,2019-07-30 13:35:21,6099,2019-07-30 13:42:15,7066,414,1,Ste-Catherine / Labelle,45.515,-73.559,St-Urbain / de la Gauchetière,...,0,1,0,2019,7,30,13,35,21,48921.000


### Adding number of trips in a +/- 30 minutes time frame of that particular station

This will serve as our "y" parameter, aka the value we are trying to predict

In [15]:
# _m = bixi_2019_all_with_stations_and_weather['start_month'] == 8
# _d = bixi_2019_all_with_stations_and_weather['start_day'] == 14
# _h = bixi_2019_all_with_stations_and_weather['start_hour'] == 8
# _c = bixi_2019_all_with_stations_and_weather['start_station_code'] == 6176
# _c2 = bixi_2019_all_with_stations_and_weather['start_station_code'] == 6014

# temp = bixi_2019_all_with_stations_and_weather[_m & _d & (_c | _c2)].sample(10)

#2019-09-22 15:29:16 	6152
#2019-04-14 08:49:42 	6176
        
temp = bixi_2019_all_with_stations_and_weather
len(temp)

5442288

Slow. 500+ hours to run

In [16]:
# # I couldn't figure out how to vectorize this...

# with tqdm(total=len(temp)) as pbar: # Progress bar
#     for i, row in temp.iterrows():
#         # Get the current row's station code
#         target_station_code = row['start_station_code']
#         target_start_date = row['start_date']

#         # Retrieve all the data for that particular station code
#         df_station = temp[(temp['start_station_code'] == target_station_code)]
        
#         # Count how many trips were started +/- 30min of the start time
#         sum_count = df_station['start_date'].between(
#             target_start_date + pd.Timedelta('-30 minutes'),
#             target_start_date + pd.Timedelta('30 minutes')
#         ).sum()

#         #Assign sum to new trip_count column
#         temp.loc[i,'trip_count'] = sum_count
        
#         #Update progress bar
#         pbar.update(1)

# temp.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')

In [17]:
# processed = pd.read_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
# processed.sort_values(by='start_date')

A little bit better but still slow. Maybe 250 hours?

In [18]:
# def count_trips(row):
#     station = temp['start_station_code'] == row['start_station_code']
#     t_date = row['start_date']
#     t_start = row['start_date'] + pd.Timedelta('-30 minutes')
#     t_end = row['start_date'] + pd.Timedelta('30 minutes')
#     return len(temp[station & (t_start < t_date) & (t_date < t_end)])

In [19]:
# start_time = time.time()
# temp['trip_count'] = temp.apply(count_trips, axis=1)
# print("--- %s seconds ---" % (time.time() - start_time))
# temp.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')

# temp.sort_values(by='start_date')

In [20]:
# processed = pd.read_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
# processed.sort_values(by='start_date')

Better - I think we can optimize it even more but that's good enough. Run time ~ 1 hour

In [21]:
# group = temp.groupby('start_station_code')

# with tqdm(total=len(group)) as pbar: # Progress bar
#     trip_count_temp = []
#     for code, df in group:
#         trip_count_group = df.apply(lambda x: 
#                                     df['start_date'].between(
#                                         x['start_date'] + pd.Timedelta('-30 minutes'),
#                                         x['start_date'] + pd.Timedelta('30 minutes')).sum(), axis=1)
#         trip_count_temp.append(trip_count_group)
#         pbar.update(1)

# print("Saving trip_count_temp to temp file...")
# np.save('./data/trip_count_temp.npy', trip_count_temp)
# print("Done")


# # DO NOT RUN BELOW IF PROCESSING ALL THE DATA. IT IS LIKELY TO THROW AN OUT OF MEMORY ERROR


# print("Converting... This may take a while but it should be faster than the previous one")
# # Throws a out of memory error if processing 5 million + data with 32GB ram
# trip_count = pd.DataFrame(pd.DataFrame(trip_count_temp).T.apply(lambda x: x.sum(), axis=1), columns=['trip_count'])
# print("Done")

# trip_count

# print("Concat to temp...")
# tc = pd.concat([temp, trip_count], axis=1, sort=False)
# print("Done")

# print("Saving to disk...")
# tc.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
# print("Done")

In [22]:
trip_count_temp_file = np.load('./data/trip_count_temp.npy', allow_pickle=True)

Merging series together

In [23]:
trip_count_series = pd.Series()

with tqdm(total=len(trip_count_temp_file)) as pbar: # Progress bar
    for series in trip_count_temp_file:
        trip_count_series = trip_count_series.append(series)
        pbar.update(1)

print(trip_count_series.shape)
trip_count_series = trip_count_series.sort_index()
trip_count_series.describe()

<ipython-input-23-d08a9357df59>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  trip_count_series = pd.Series()
100%|██████████| 619/619 [00:22<00:00, 27.09it/s] 


(5442288,)


count   5442288.000
mean          7.828
std           8.266
min           1.000
25%           3.000
50%           6.000
75%          10.000
max         223.000
dtype: float64

Merge trip count data to overall data

In [24]:
temp = pd.concat([temp, trip_count_series], axis=1, sort=False)

In [25]:
temp = temp.rename({0: "trip_count"}, axis=1)

In [26]:
temp.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Rain Showers,Thunderstorms,start_year,start_month,start_day,start_hour,start_minute,start_second,start_seconds_since_midnight,trip_count
1025480,2019-06-21 10:11:26,7109,2019-06-21 10:13:28,7109,122,1,Cégep André-Laurendeau,45.436,-73.606,Cégep André-Laurendeau,...,0,0,2019,6,21,10,11,26,36686.000,2
1961074,2019-05-17 23:54:49,6018,2019-05-18 00:02:27,6132,458,1,St-André / Ontario,45.519,-73.564,Larivière / de Lorimier,...,0,0,2019,5,17,23,54,49,86089.000,4
1048538,2019-07-14 12:31:20,6175,2019-07-14 12:36:50,6211,330,1,St-Christophe / Cherrier,45.520,-73.568,Roy / St-Laurent,...,0,0,2019,7,14,12,31,20,45080.000,2
2582213,2019-04-29 15:49:37,6732,2019-04-29 15:54:41,10002,304,1,Fortune / Wellington,45.478,-73.559,Métro Charlevoix (Centre / Charlevoix),...,0,0,2019,4,29,15,49,37,56977.000,3
2982686,2019-05-05 12:32:15,6114,2019-05-05 12:52:46,6023,1231,0,Métro Papineau (Cartier / Ste-Catherine),45.524,-73.552,de la Commune / Berri,...,0,0,2019,5,5,12,32,15,45135.000,9


In [27]:
bixi_2019_all_with_stations_and_weather_and_trip_count = temp

## Preparing data for training

In [28]:
bixi_2019_all_with_stations_and_weather_and_trip_count.columns

Index(['start_date', 'start_station_code', 'end_date', 'end_station_code',
       'duration_sec', 'is_member', 'start_name', 'start_latitude',
       'start_longitude', 'end_name', 'end_latitude', 'end_longitude',
       'start_weather', 'start_temp', 'Clear', 'Cloudy', 'Drizzle', 'Fog',
       'Heavy Rain Showers', 'Mainly Clear', 'Moderate Rain',
       'Moderate Rain Showers', 'Mostly Cloudy', 'Rain', 'Rain Showers',
       'Thunderstorms', 'start_year', 'start_month', 'start_day', 'start_hour',
       'start_minute', 'start_second', 'start_seconds_since_midnight',
       'trip_count'],
      dtype='object')

In [29]:
bixi_train_data = bixi_2019_all_with_stations_and_weather_and_trip_count[[
    'start_station_code',
    'start_temp',
    'Clear', 
    'Cloudy', 
    'Drizzle', 
    'Fog',
    'Heavy Rain Showers', 
    'Mainly Clear', 
    'Moderate Rain',
    'Moderate Rain Showers', 
    'Mostly Cloudy', 
    'Rain', 
    'Rain Showers',
    'Thunderstorms', 
    'start_year', 
    'start_month', 
    'start_day', 
    'start_hour',
    'start_minute', 
    'start_second',
    'trip_count' #Will get separated later
]]
bixi_train_data.sample(5)

,start_station_code,start_temp,Clear,Cloudy,Drizzle,Fog,Heavy Rain Showers,Mainly Clear,Moderate Rain,Moderate Rain Showers,...,Rain,Rain Showers,Thunderstorms,start_year,start_month,start_day,start_hour,start_minute,start_second,trip_count
3910735,7141,10.300,0,0,0,0,0,0,0,0,...,0,0,0,2019,5,21,8,6,3,4
1368264,6738,14.500,0,1,0,0,0,0,0,0,...,0,0,0,2019,10,9,16,13,2,8
1010417,6726,16.900,0,0,0,0,0,0,0,0,...,0,0,0,2019,6,9,2,18,5,1
5199218,6043,16.600,0,0,0,0,0,0,0,0,...,0,0,0,2019,6,14,10,4,35,3
1045386,6052,24.200,0,0,0,0,0,0,0,0,...,0,0,0,2019,7,7,14,25,25,29


In [30]:
bixi_train_data.describe()

,start_temp,Clear,Cloudy,Drizzle,Fog,Heavy Rain Showers,Mainly Clear,Moderate Rain,Moderate Rain Showers,Mostly Cloudy,Rain,Rain Showers,Thunderstorms,start_year,start_month,start_day,start_hour,start_minute,start_second,trip_count
count,5440794.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000
mean,18.999,0.052,0.056,0.006,0.011,0.000,0.074,0.001,0.001,0.116,0.035,0.021,0.003,2019.000,7.240,16.040,14.126,29.413,29.495,7.828
std,6.344,0.221,0.230,0.078,0.104,0.010,0.261,0.031,0.029,0.321,0.183,0.144,0.051,0.000,1.705,8.887,5.197,17.253,17.319,8.266
min,-2.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2019.000,4.000,1.000,0.000,0.000,0.000,1.000
25%,14.400,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2019.000,6.000,8.000,10.000,14.000,14.000,3.000
50%,19.300,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2019.000,7.000,16.000,15.000,29.000,30.000,6.000
75%,24.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2019.000,9.000,24.000,18.000,44.000,44.000,10.000
max,32.800,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,2019.000,10.000,31.000,23.000,59.000,59.000,223.000


### Drop rows with null values

In [31]:
bixi_train_data.isnull().sum()

start_station_code          0
start_temp               1494
Clear                       0
Cloudy                      0
Drizzle                     0
Fog                         0
Heavy Rain Showers          0
Mainly Clear                0
Moderate Rain               0
Moderate Rain Showers       0
Mostly Cloudy               0
Rain                        0
Rain Showers                0
Thunderstorms               0
start_year                  0
start_month                 0
start_day                   0
start_hour                  0
start_minute                0
start_second                0
trip_count                  0
dtype: int64

In [32]:
bixi_train_data = bixi_train_data.dropna()
bixi_train_data.isnull().sum()

start_station_code       0
start_temp               0
Clear                    0
Cloudy                   0
Drizzle                  0
Fog                      0
Heavy Rain Showers       0
Mainly Clear             0
Moderate Rain            0
Moderate Rain Showers    0
Mostly Cloudy            0
Rain                     0
Rain Showers             0
Thunderstorms            0
start_year               0
start_month              0
start_day                0
start_hour               0
start_minute             0
start_second             0
trip_count               0
dtype: int64

In [33]:
bixi_train_data.columns

Index(['start_station_code', 'start_temp', 'Clear', 'Cloudy', 'Drizzle', 'Fog',
       'Heavy Rain Showers', 'Mainly Clear', 'Moderate Rain',
       'Moderate Rain Showers', 'Mostly Cloudy', 'Rain', 'Rain Showers',
       'Thunderstorms', 'start_year', 'start_month', 'start_day', 'start_hour',
       'start_minute', 'start_second', 'trip_count'],
      dtype='object')

## Splitting data by start station code
We will train a model for each station

In [34]:
train_group = bixi_train_data.groupby('start_station_code')

In [35]:
train_group_data = {}

with tqdm(total=len(train_group)) as pbar:
    for key, df in train_group:
        X_train = df[['start_temp', 'Clear', 'Cloudy', 'Drizzle', 'Fog',
                      'Heavy Rain Showers', 'Mainly Clear', 'Moderate Rain','Moderate Rain Showers', 
                      'Mostly Cloudy', 'Rain', 'Rain Showers', 'Thunderstorms', 'start_year',
                      'start_month', 'start_day', 'start_hour','start_minute', 'start_second']]
        y_train = df[['trip_count']]

        train_group_data[key] = {
            "X_train": X_train,
            "y_train": y_train
        }
        pbar.update(1)

100%|██████████| 619/619 [00:04<00:00, 145.29it/s]


Data can be accessed using:
train_group_data[4000]['y_train']

# Training Models

In [36]:
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

# Linear Regression

In [37]:
models_lr = {}
models_lr_r2score = {}

with tqdm(total=len(train_group_data)) as pbar:
    for key in train_group_data:
        # Training data
        X_train = train_group_data[key]['X_train']
        y_train = train_group_data[key]['y_train'].to_numpy().flatten()
        
        # Model
        model = LinearRegression().fit(X_train, y_train)
        models_lr[key] = model
        
        # Training error
        models_lr_r2score[key] = model.score(X_train, y_train)
        
        pbar.update(1)

100%|██████████| 619/619 [00:04<00:00, 152.77it/s]


In [38]:
df_models_lr_r2score = pd.DataFrame.from_dict(models_lr_r2score, orient='index')
df_models_lr_r2score.describe()

,0
count,619.000
mean,0.110
std,0.054
min,0.020
25%,0.068
50%,0.102
75%,0.139
max,0.409


From above, we see that the score is quite low for Linear Regression (11%)

## SGD Regression

In [39]:
models_sgdr = {}
models_sgdr_r2score = {}

with tqdm(total=len(train_group_data)) as pbar:
    for key in train_group_data:
        # Training data
        X_train = train_group_data[key]['X_train']
        y_train = train_group_data[key]['y_train'].to_numpy().flatten()
        
        #Normalize data
        flow = make_pipeline(MinMaxScaler(), SGDRegressor())
        
        # Model
        model = flow.fit(X_train, y_train)
        models_sgdr[key] = model
        
        # Training error
        models_sgdr_r2score[key] = model.score(X_train, y_train)
        
        pbar.update(1)

100%|██████████| 619/619 [00:17<00:00, 35.24it/s]


In [40]:
df_models_sgdr_r2score = pd.DataFrame.from_dict(models_sgdr_r2score, orient='index')
df_models_sgdr_r2score.describe()

,0
count,619.000
mean,0.103
std,0.056
min,-0.195
25%,0.063
50%,0.097
75%,0.134
max,0.376


## MLP Regressor

In [41]:
models_mlpr = {}
models_mlpr_r2score = {}

with tqdm(total=len(train_group_data)) as pbar:
    for key in train_group_data:
        # Training data
        X_train = train_group_data[key]['X_train']
        y_train = train_group_data[key]['y_train'].to_numpy().flatten()

        # Model
        model = MLPRegressor(max_iter=1000).fit(X_train, y_train)
        models_mlpr[key] = model
        
        # Training Error
        models_mlpr_r2score[key] = model.score(X_train, y_train)
        
        pbar.update(1)

100%|██████████| 619/619 [13:25<00:00,  1.30s/it]  


In [42]:
df_models_mlpr_r2score = pd.DataFrame.from_dict(models_mlpr_r2score, orient='index')
df_models_mlpr_r2score.describe()

,0
count,619.000
mean,-40.721
std,422.689
min,-5781.154
25%,0.034
50%,0.072
75%,0.119
max,0.685
